In [ ]:
import GlobalVariables as gv
import pygui as pgui
import pandas as pd
pgui.setup_interface()
pgui.db_list.value = 'tmind'
event = {'name': 'row_added', 'index': 1, 'source': 'gui'}

# Make a new fow for the existing dataframe
df = pgui.grid_manager.filterGridObject.qgrid_filter.get_changed_df()
pgui.grid_manager.filterGridObject.selection_widgets['Data Set'].value = 'Multi Index'
pgui.grid_manager.filterGridObject.selection_widgets['Creature'].value = ['Necromancer']

new_row = pd.DataFrame({
            'Modifier': [''],
            'op1': [''],
            'Creature': [''],
            'op2': [''],
            'Spell': ['River Of Souls'],
            'Data Set': ['Multi Index'],
            'Active': [True]
        })
new_df = pd.concat([df, new_row], ignore_index=True)
pgui.grid_manager.filterGridObject.qgrid_filter.df = new_df
pgui.grid_manager.filterGridObject.grid_filter_on_row_added(event, pgui.grid_manager.filterGridObject.qgrid_filter)

In [1]:
import pygui as pgui
#pgui.username.value = 'tmind'
#change = {'name': 'value', 'old': '', 'new': 'tmind'}
#pgui.handle_db_list_change(change)
#pgui.reload_data_on_click(None,'Decks')
pgui.db_list.value = 'tmind'

#GridManager.get_forgeborn_abilities()

#event = {'name': 'row_added', 'index': 1, 'source': 'gui'}
#pgui.grid_manager.filterGridObject.grid_filter_on_row_added(event, pgui.grid_manager.filterGridObject.qgrid_filter)
#pgui.reload_data_on_click(None,'Decks')
if False:
    #pgui.db_list.value = 'tmind'
    #event = {'name': 'selection_changed', 'old': [], 'new': [1], 'source': 'gui'}

    event = {'name': 'cell edited', 'index': 0, 'column': 'Active', 'new': False, 'source': 'filter'}
    pgui.grid_manager.filterGridObject.grid_filter_on_cell_edit(event,pgui.grid_manager.filterGridObject.qgrid_filter)
    #collection_grid = pgui.qm.grids.get('collection')
    #df = pgui.generate_deck_content_dataframe(event, collection_grid.main_widget)
    #df = pgui.generate_deck_statistics_dataframe()

NameError: name 'pgui' is not defined

In [ ]:
# Test creation of fusions 

import pygui as pgui
pgui.setup_interface()
pgui.db_list.value = 'tmind'
pgui.reload_data_on_click(pgui.button_load, 'Fusions')